## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Walvis Bay,NaN,-22.96,14.51,55.99,87,0,5.82,clear sky
1,1,Chokurdakh,RU,70.63,147.92,34.68,88,100,10.65,overcast clouds
2,2,Fairbanks,US,64.84,-147.72,64.40,68,90,1.50,overcast clouds
3,3,Luganville,VU,-15.53,167.17,77.00,100,75,7.16,broken clouds
4,4,Ostrovnoy,RU,68.05,39.51,58.14,90,32,16.96,scattered clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,Luganville,VU,-15.53,167.17,77.00,100,75,7.16,broken clouds
6,6,Hasaki,JP,35.73,140.83,75.20,100,75,4.70,mist
7,7,Puerto Ayora,EC,-0.74,-90.35,77.00,53,63,17.22,broken clouds
9,9,Faanui,PF,-16.48,-151.75,76.64,78,16,19.53,few clouds
10,10,Victoria,HK,22.29,114.16,84.00,79,1,8.99,clear sky
12,12,Isangel,VU,-19.55,169.27,79.48,81,16,4.92,few clouds
13,13,Avarua,CK,-21.21,-159.78,75.20,73,2,11.41,clear sky
17,17,Juegang,CN,32.32,121.19,75.65,91,0,7.20,clear sky
22,22,Atuona,PF,-9.80,-139.03,78.44,77,1,15.75,clear sky
27,27,San Cristobal,VE,7.77,-72.22,87.80,45,40,19.46,scattered clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                1
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_preferred_cities_df = preferred_cities_df.dropna()
clean_preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Luganville,VU,77.00,broken clouds,-15.53,167.17,
6,Hasaki,JP,75.20,mist,35.73,140.83,
7,Puerto Ayora,EC,77.00,broken clouds,-0.74,-90.35,
9,Faanui,PF,76.64,few clouds,-16.48,-151.75,
10,Victoria,HK,84.00,clear sky,22.29,114.16,
12,Isangel,VU,79.48,few clouds,-19.55,169.27,
13,Avarua,CK,75.20,clear sky,-21.21,-159.78,
17,Juegang,CN,75.65,clear sky,32.32,121.19,
22,Atuona,PF,78.44,clear sky,-9.80,-139.03,
27,San Cristobal,VE,87.80,scattered clouds,7.77,-72.22,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # 6e. Make request and retrieve the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Luganville,VU,77.00,broken clouds,-15.53,167.17,The Espiritu Hotel
6,Hasaki,JP,75.20,mist,35.73,140.83,Inubosaki Kanko Hotel
7,Puerto Ayora,EC,77.00,broken clouds,-0.74,-90.35,Finch Bay Galapagos Hotel
9,Faanui,PF,76.64,few clouds,-16.48,-151.75,Bora-Bora Pearl Beach Resort & Spa
10,Victoria,HK,84.00,clear sky,22.29,114.16,Mini Hotel Central


In [10]:
hotel_df.isnull().sum()

City                   0
Country                0
Max Temp               0
Current Description    0
Lat                    0
Lng                    0
Hotel Name             0
dtype: int64

In [11]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df 

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Luganville,VU,77.00,broken clouds,-15.53,167.17,The Espiritu Hotel
6,Hasaki,JP,75.20,mist,35.73,140.83,Inubosaki Kanko Hotel
7,Puerto Ayora,EC,77.00,broken clouds,-0.74,-90.35,Finch Bay Galapagos Hotel
9,Faanui,PF,76.64,few clouds,-16.48,-151.75,Bora-Bora Pearl Beach Resort & Spa
10,Victoria,HK,84.00,clear sky,22.29,114.16,Mini Hotel Central
...,...,...,...,...,...,...,...
688,Salalah,OM,77.00,overcast clouds,17.02,54.09,HAMDAN PLAZA HOTEL
689,Makakilo City,US,86.00,broken clouds,21.35,-158.09,Marriott's Ko Olina Beach Club
693,Virginia Beach,US,90.00,broken clouds,36.85,-75.98,Hilton Virginia Beach Oceanfront
694,Huilong,CN,82.99,clear sky,31.81,121.66,Hilton Garden Inn Qidong


In [12]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [18]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>{Max Temp} °F</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [19]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))